In [ ]:
# https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1 を参照
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3==1.0.3 > /dev/null
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!pip install transformers
!pip install jaconv
!pip install unicodedata
!pip install neologdn

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
     |████████████████████████████████| 3.1 MB 28.1 MB/s 
     |████████████████████████████████| 3.3 MB 44.7 MB/s 
     |████████████████████████████████| 596 kB 7.4 MB/s 
     |████████████████████████████████| 895 kB 29.0 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15565 sha256=8ca788b85055e000dee21c5cdad3d23c1b9b21153188c090b9df6202bda55b75
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
Successfully built jaconv
ERROR: C

In [16]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("drive/MyDrive/Joint_Inference")

Mounted at /content/drive


In [ ]:
# フォルダを作成
! mkdir resource BERT
# 万病辞書のダウンロード
! wget -O ./resource/MANBYO_201907_Dic-utf8.dic  http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic 
# UTH-BERTのダウンロード
! wget -O ./BERT/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K.zip https://ai-health.m.u-tokyo.ac.jp/labweb/dl/uth_bert/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K.zip
# ファイルを解凍
! unzip ./BERT/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K.zip -d ./BERT
# モデルのダウンロード
! wget --load-cookies /tmp/cookies.txt "https://drive.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1Ec8P255DFB1KJkaZYO_MZAyLE9CXxrp6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Ec8P255DFB1KJkaZYO_MZAyLE9CXxrp6" -O ./models/20211012/model.pt && rm -rf /tmp/cookies.txt

--2021-11-05 02:41:30--  http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic
Resolving sociocom.jp (sociocom.jp)... 49.212.199.94
Connecting to sociocom.jp (sociocom.jp)|49.212.199.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71728213 (68M)
Saving to: ‘./resource/MANBYO_201907_Dic-utf8.dic’

./resource/MANBYO_2 100%[===================>]  68.41M  12.9MB/s    in 6.3s    

2021-11-05 02:41:36 (10.9 MB/s) - ‘./resource/MANBYO_201907_Dic-utf8.dic’ saved [71728213/71728213]

--2021-11-05 02:41:37--  https://ai-health.m.u-tokyo.ac.jp/labweb/dl/uth_bert/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K.zip
Resolving ai-health.m.u-tokyo.ac.jp (ai-health.m.u-tokyo.ac.jp)... 163.43.80.52
Connecting to ai-health.m.u-tokyo.ac.jp (ai-health.m.u-tokyo.ac.jp)|163.43.80.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922212539 (1.8G) [application/zip]
Saving to: ‘./BERT/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K.zip’

./BERT/UTH_BERT_B

In [ ]:
from lib.models import BERT_JOINT
from lib.util import load_model, decode_ner, decode_re, spacy2df
from lib.preprocess import uth_bert_preprocess, mecab_wakati, make_vector
import os, sys, random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from pathlib import Path
from transformers import BertModel, BertTokenizer
import spacy
from spacy import displacy
from lib.html import transform_result2html_re, transform_result2html_ner
from IPython.display import HTML
from dataclasses import dataclass

In [ ]:
# https://github.com/WindChimeRan/pytorch_multi_head_selection_re を参照
@dataclass
class Parameters(object):
  bert_path: str = "./BERT/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K"
  neologd_path: str = "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
  manbyo_path: str = "./resource/MANBYO_201907_Dic-utf8.dic"
  rel2idx_path: str = "./models/20211012/rel2idx.json"
  tag2idx_path: str = "./models/20211012/tag2idx.json"
  model_path: str = "./models/20211012/model.pt"
  max_text_len: int = 510

In [ ]:
# 情報抽出を行う文字列
text = "患者:55歳,男性.事務員.\n主訴:嘔気.\n既往歴:特記すべきことなし."
text = "患者：69歳，男性。\n主　訴：食思不振，腰痛。\n現病歴：2型糖尿病に対してメトホルミンの内服をしていた。"

In [ ]:
paras = Parameters()
text = uth_bert_preprocess(text)
tokens = mecab_wakati(text, paras)
vecs = make_vector(tokens, paras)
model, tag2idx, rel2idx = load_model(paras)
ner_preds, re_preds = model(torch.tensor(vecs), rel2idx)
ner_preds = decode_ner(ner_preds, tag2idx)
spacy_format = decode_re(tokens, re_preds, ner_preds, rel2idx)
res_df = spacy2df(spacy_format, "result.csv")
re_htmls = transform_result2html_re(res_df, spacy_format)
ner_htmls = transform_result2html_ner(res_df)

Some weights of the model checkpoint at ./BERT/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/content/drive/My Drive/UTH-17_Joint_Inference/lib/models/torchcrf_mod.py:360: UserWarning: w

In [ ]:
HTML(ner_htmls)

Score,Text
,person患者 ： age６ ９ 歳 、 sex男性 。 CC主訴 ： state食思不振 、 state腰痛 。 PresentHistory現 病歴 ： state２型糖尿病 に対して drugメトホルミン の treatment内服 を execute_Doneし て い た 。
,


In [ ]:
HTML(re_htmls)

Score,Relation,Text (Head - Tail)
,subject,person患者 ： age６ ９ 歳 、 男性 。 主訴 ： 食思不振 、 腰痛 。 現 病歴 ： ２型糖尿病 に対して メトホルミン の 内服 を し て い た 。
,subject,person患者 ： ６ ９ 歳 、 sex男性 。 主訴 ： 食思不振 、 腰痛 。 現 病歴 ： ２型糖尿病 に対して メトホルミン の 内服 を し て い た 。
,category,患者 ： ６ ９ 歳 、 男性 。 CC主訴 ： state食思不振 、 腰痛 。 現 病歴 ： ２型糖尿病 に対して メトホルミン の 内服 を し て い た 。
,category,患者 ： ６ ９ 歳 、 男性 。 CC主訴 ： 食思不振 、 state腰痛 。 現 病歴 ： ２型糖尿病 に対して メトホルミン の 内服 を し て い た 。
,executed,患者 ： ６ ９ 歳 、 男性 。 主訴 ： 食思不振 、 腰痛 。 現 病歴 ： ２型糖尿病 に対して drugメトホルミン の treatment内服 を し て い た 。
,category,患者 ： ６ ９ 歳 、 男性 。 主訴 ： 食思不振 、 腰痛 。 PresentHistory現 病歴 ： ２型糖尿病 に対して メトホルミン の 内服 を execute_Doneし て い た 。
,reason,患者 ： ６ ９ 歳 、 男性 。 主訴 ： 食思不振 、 腰痛 。 現 病歴 ： state２型糖尿病 に対して メトホルミン の 内服 を execute_Doneし て い た 。
,executed,患者 ： ６ ９ 歳 、 男性 。 主訴 ： 食思不振 、 腰痛 。 現 病歴 ： ２型糖尿病 に対して メトホルミン の treatment内服 を execute_Doneし て い た 。
,,


In [ ]:
res_df

,index,word,IOB,rel_type,tail_index
0,0,患者,B-person,None,None
1,1,：,O,None,None
2,2,６,B-age,subject,0
3,3,９,I-age,None,None
4,4,歳,I-age,None,None
5,5,、,O,None,None
6,6,男性,B-sex,subject,0
7,7,。,O,None,None
8,8,主訴,B-CC,None,None
9,9,：,O,None,None
